In [8]:
# Import the necessary modules

import datetime as dt
import time
import smtplib

from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from dateutil.relativedelta import relativedelta

""" datetime util """
now = dt.datetime.now()
lastday = now + relativedelta(months=0, days=-1)
firstday_of_this_month = dt.datetime(now.year, now.month, 1)
lastday_of_this_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=1, days=-1)
firstday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=-1, days=0)
lastday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=0, days=-1)

def last_working_day(given_date):
    while True:
        given_date -= dt.timedelta(days=1)
        if given_date.weekday() not in [5,6]:
            return given_date
        
""" set the download window """
start_date = "2017-01-01"
end_date = last_working_day(now).strftime('%Y-%m-%d')

# Define the default_args for the DAG
default_args = {
    'owner': 'you',
    'depends_on_past': False,
    'start_date': end_date,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': dt.timedelta(minutes=1),
}

# Define the DAG
dag = DAG(
    'parallel_tasks',
    default_args=default_args,
    description='Two parallel tasks: counting in 10 minutes and sending email every minute',
    schedule_interval=dt.timedelta(minutes=1),
)

# Define the first task, counting in 10 minutes
def count_in_10_minutes():
    for i in range(10):
        print(f'{i + 1} minutes have passed.')
        time.sleep(60)

count_in_10_minutes_task = PythonOperator(
    task_id='count_in_10_minutes',
    python_callable=count_in_10_minutes,
    dag=dag,
)

# Define the second task, sending email every minute
def send_email_every_minute():
    # Define the SMTP server and login credentials
    smtp_server = "smtp.gmail.com"
    port = 587
    login = "youremail@gmail.com"
    password = "yourpassword"

    # Connect to the SMTP server
    server = smtplib.SMTP(smtp_server, port)
    server.ehlo()
    server.starttls()
    server.login(login, password)

    # Define the recipient email address and subject
    to = "recipient@email.com"
    subject = "Minutely Email"

    # Send the email every minute
    for i in range(10):
        message = f"This is minute {i + 1} of the email sequence."
        msg = f"Subject: {subject}\n\n{message}"
        server.sendmail(login, to, msg)
        time.sleep(60)

    # Close the SMTP connection
    server.quit()

send_email_every_minute_task = PythonOperator(
    task_id='send_email_every_minute',
    python_callable=send_email_every_minute,
    dag=dag,
)

# Set the task order
count_in_10_minutes_task >> send_email_every_minute_task


C:\Users\speed\AppData\Local\Temp\ipykernel_19188\3729974898.py:8 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

C:\Users\speed\AppData\Local\Temp\ipykernel_19188\3729974898.py:41 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

<Task(PythonOperator): send_email_every_minute>